In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../utils')
sys.path.append('../templates')

from common_lib import check_packages
check_packages()


Proceed.


In [2]:
import os
import json
import boto3
import torch
import sagemaker, boto3, jinja2
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker.estimator import Estimator
from sagemaker import serializers, deserializers
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


<br>

## 1. Local에서 학습한 PEFT 모델을 로드
---

In [3]:
%store -r output_dir

In [4]:
local_peft_model_dir = output_dir

In [5]:
local_peft_model_dir

'output'

In [ ]:
torch.cuda.empty_cache()

### Model 환경설정

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [13]:
from peft import PeftModel, PeftConfig

In [14]:
peft_model_id = local_peft_model_dir
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()
     

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                

### 로컬 모델 호출 테스트

In [41]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q, 
            return_tensors='pt', 
            return_token_type_ids=False
        ).to('cuda'), 
        max_new_tokens=512,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [42]:
gen('프로그래밍을 잘하기 위한 세 가지 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 프로그래밍을 잘하기 위한 세 가지 방법은?

### 답변:
1. # 연습, 연습, 또 연습: 연습을 하지 않으면 어떤 것도 숙달할 수 없습니다. 좋은 프로그래머가 되기 위해서는 연습이 필수적입니다. 코딩을 하면서 항상 문제를 해결하고, 새로운 문제나 솔루션을 생각하며, 자신이 작성한 코드를 다른 프로그래머에게 보여주는 연습을 하세요. 실력을 향상하기 위해 사용할 수 있는 또 다른 방법은 코딩 경험을 쌓을 수 있는 프로젝트를 작성하는 것입니다. 실제 상황에서 프로그래밍에 참여하여 실제 문제를 만나고, 이를 해결하는 연습을 하세요.2. 코드 정리하기: 소프트웨어 개발은 문제나 이슈가 발생하기 쉬운 매우 난해하고 복잡한 작업입니다. 오류와 누락된 코드를 식별하기 위해 코딩 습관을 모니터링하는 것은 프로그래머의 성공에 매우 중요합니다. 자신의 코드를 정기적으로 검토하고, 오류와 중복 코드를 찾아내고, 필요한 경우 코드를 재작성하는 것을 고려하세요. 모든 소프트웨어 개발은 버그와 에러가 발견될 수 있기에 코드가 효율적이고 정확한지 확인하기 위해 코드를 정기적으로 검토하는 것이 중요합니다.3. 적극적인 학습자가 되십시오: 다른 사람으로부터 배우고, 다른 사람의 코딩을 검토하고, 그 과정에서 스스로에게도 배울 때 진정한 실력을 쌓을 수 있습니다. 새로운 기술과 트렌드에 항상 열려 있어야 하므로, 새로운 코드 작성 방법과 새로운 기술 및 기법을 항상 배우세요. 동료, 선생님 또는 기술을 향상시키는 데 도움이 될 수 있는 다른 사람에게 질문하는 것을 두려워하지 마세요.항상 새로운 문제를 기꺼이 해결하고 다른 사람들로부터 끊임없이 배우기 위해 노력하세요. 이를 통해 최고의 프로그래머가 되기 위한 강력한 기술과 지식을 개발할 수 있습니다.<|endoftext|>


## SageMaker Endpoint를 위한 함수 처리 테스트

In [76]:

import os.path as osp
import json
def load_template(template_name: str):
    file_name = osp.join("../templates", f"{template_name}.json")
    if not osp.exists(file_name):
        raise ValueError(f"Can't read {file_name}")
    with open(file_name, encoding='utf-8') as fp:  # 'utf-8' 인코딩으로 파일을 엽니다.
        return json.load(fp)


def generate_prompt(template, instruction: str, input_text: Union[None, str] = None, label: Union[None, str] = None) -> str:
    if input_text:
        res = template["prompt_input"].format(instruction=instruction, input=input_text)
    else:
        res = template["prompt_no_input"].format(instruction=instruction)
    if label:
        res = f"{res}{label}"
    return res

def get_payload(instruction, input_text, params, template_name="kullm"):
    
    template = load_template(template_name)
    prompt = generate_prompt(template, instruction, input_text)
    payload = {
        'inputs': prompt,
        'parameters': params
    }
    payload_str = json.dumps(payload)
    return payload_str.encode("utf-8")

def infer(self, payload, content_type="application/json", verbose=True):
    response = self.smr_client.invoke_endpoint(
        EndpointName=self.endpoint_name,
        ContentType=content_type,
        Body=payload
    )

    res = json.loads(response['Body'].read().decode("utf-8"))
    generated_text = res[0]["generated_text"]
    #generated_text = self.prompter.get_response(generated_text)

    generated_text = generated_text.split('###')[0]
    if verbose:
        pprint.pprint(f'Response: {generated_text}')
    return generated_text
def parse_response(query_response):
    
    def traverse(o, tree_types=(list, tuple)):
        if isinstance(o, tree_types):
            for value in o:
                for subvalue in traverse(value, tree_types):
                    yield subvalue
        else:
            yield o

    data = eval(query_response)
    
    listRes = []
    for value in traverse(data):
        listRes.append(value["generated_text"])
        
    if len(listRes) >= 2: return listRes
    else: return listRes[0].strip()

In [77]:
params = {
    "do_sample": True,
    "max_new_tokens": 512,
    "temperature": 0.7,
    "top_p": 0.9,
    "return_full_text": False,
    "repetition_penalty": 1.1,
    "presence_penalty": None,
    "eos_token_id": 2,
}
instruction = "다음 글을 알기 쉽게 요약해 주세요."
context = """
대규모 언어 모델(LLM; Large Language Models) 분야의 발전과 LLM이 가치 있는 인사이트를 제공하는 문제 세트 수가 계속 증가하고 있다는 소식을 들어보셨을 겁니다. 
대규모 데이터 세트와 여러 작업을 통해 훈련된 대규모 모델은 훈련되지 않은 특정 작업에도 잘 일반화됩니다. 이러한 모델을 파운데이션 모델(foundation model)이라고 하며, 스탠포드 HAI 연구소(Stanford Institute for Human-Centered Artificial Intelligence)에서 처음 대중화한 용어입니다. 
이러한 파운데이션 모델은 프롬프트 엔지니어링(prompt engineering) 기법의 도움으로 잘 활용할 수 있지만, 유스케이스가 도메인에 매우 특화되어 있거나 작업의 종류가 매우 다양하여 모델을 추가로 커스터마이징해야 하는 경우가 많습니다. 
특정 도메인이나 작업에 대한 대규모 모델의 성능을 개선하기 위한 한 가지 접근 방식은 더 작은 작업별 데이터 세트로 모델을 추가로 훈련하는 것입니다. 파인 튜닝(fine-tuning)으로 알려진 이 접근 방식은 LLM의 정확도를 성공적으로 개선하지만, 모든 모델 가중치를 수정해야 합니다. 
파인 튜닝 데이터 세트 크기가 훨씬 작기 때문에 사전 훈련(pre-training)하는 것 보다 훨씬 빠르지만 여전히 상당한 컴퓨팅 성능과 메모리가 필요합니다. 
파인 튜닝은 원본 모델의 모든 파라미터 가중치를 수정하므로 비용이 많이 들고 원본 모델과 동일한 크기의 모델을 생성하므로 스토리지 용량에도 부담이 됩니다.
"""
payload = get_payload(instruction, context, params, "kullm")

In [78]:
data = json.loads(payload.decode('utf-8'))
input_prompt, params = data["inputs"], data["parameters"]

In [79]:
data

{'inputs': '아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n다음 글을 알기 쉽게 요약해 주세요.\n\n### 입력:\n\n대규모 언어 모델(LLM; Large Language Models) 분야의 발전과 LLM이 가치 있는 인사이트를 제공하는 문제 세트 수가 계속 증가하고 있다는 소식을 들어보셨을 겁니다. \n대규모 데이터 세트와 여러 작업을 통해 훈련된 대규모 모델은 훈련되지 않은 특정 작업에도 잘 일반화됩니다. 이러한 모델을 파운데이션 모델(foundation model)이라고 하며, 스탠포드 HAI 연구소(Stanford Institute for Human-Centered Artificial Intelligence)에서 처음 대중화한 용어입니다. \n이러한 파운데이션 모델은 프롬프트 엔지니어링(prompt engineering) 기법의 도움으로 잘 활용할 수 있지만, 유스케이스가 도메인에 매우 특화되어 있거나 작업의 종류가 매우 다양하여 모델을 추가로 커스터마이징해야 하는 경우가 많습니다. \n특정 도메인이나 작업에 대한 대규모 모델의 성능을 개선하기 위한 한 가지 접근 방식은 더 작은 작업별 데이터 세트로 모델을 추가로 훈련하는 것입니다. 파인 튜닝(fine-tuning)으로 알려진 이 접근 방식은 LLM의 정확도를 성공적으로 개선하지만, 모든 모델 가중치를 수정해야 합니다. \n파인 튜닝 데이터 세트 크기가 훨씬 작기 때문에 사전 훈련(pre-training)하는 것 보다 훨씬 빠르지만 여전히 상당한 컴퓨팅 성능과 메모리가 필요합니다. \n파인 튜닝은 원본 모델의 모든 파라미터 가중치를 수정하므로 비용이 많이 들고 원본 모델과 동일한 크기의 모델을 생성하므로 스토리지 용량에도 부담이 됩니다.\n\n\n### 응답:\n',
 'parameters': {'do_sample': True,
  'max_new_tokens': 512,
  't

In [49]:
input_prompt

'아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n다음 글을 알기 쉽게 요약해 주세요.\n\n### 입력:\n\n대규모 언어 모델(LLM; Large Language Models) 분야의 발전과 LLM이 가치 있는 인사이트를 제공하는 문제 세트 수가 계속 증가하고 있다는 소식을 들어보셨을 겁니다. \n대규모 데이터 세트와 여러 작업을 통해 훈련된 대규모 모델은 훈련되지 않은 특정 작업에도 잘 일반화됩니다. 이러한 모델을 파운데이션 모델(foundation model)이라고 하며, 스탠포드 HAI 연구소(Stanford Institute for Human-Centered Artificial Intelligence)에서 처음 대중화한 용어입니다. \n이러한 파운데이션 모델은 프롬프트 엔지니어링(prompt engineering) 기법의 도움으로 잘 활용할 수 있지만, 유스케이스가 도메인에 매우 특화되어 있거나 작업의 종류가 매우 다양하여 모델을 추가로 커스터마이징해야 하는 경우가 많습니다. \n특정 도메인이나 작업에 대한 대규모 모델의 성능을 개선하기 위한 한 가지 접근 방식은 더 작은 작업별 데이터 세트로 모델을 추가로 훈련하는 것입니다. 파인 튜닝(fine-tuning)으로 알려진 이 접근 방식은 LLM의 정확도를 성공적으로 개선하지만, 모든 모델 가중치를 수정해야 합니다. \n파인 튜닝 데이터 세트 크기가 훨씬 작기 때문에 사전 훈련(pre-training)하는 것 보다 훨씬 빠르지만 여전히 상당한 컴퓨팅 성능과 메모리가 필요합니다. \n파인 튜닝은 원본 모델의 모든 파라미터 가중치를 수정하므로 비용이 많이 들고 원본 모델과 동일한 크기의 모델을 생성하므로 스토리지 용량에도 부담이 됩니다.\n\n\n### 응답:\n'

In [52]:
result = model.generate(
        **tokenizer(
            input_prompt, 
            return_tensors='pt', 
            return_token_type_ids=False
        ).to('cuda'), 
        max_new_tokens=512,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [66]:
#print(tokenizer.decode(result[0]))
#print(result)

decoded_output = tokenizer.decode(result[0])

print("decode_output : ", decoded_output)
# 이제 JSON 형식으로 변환
json_data = json.dumps({"generated_text": decoded_output})

decode_output :  아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.

### 명령어:
다음 글을 알기 쉽게 요약해 주세요.

### 입력:

대규모 언어 모델(LLM; Large Language Models) 분야의 발전과 LLM이 가치 있는 인사이트를 제공하는 문제 세트 수가 계속 증가하고 있다는 소식을 들어보셨을 겁니다. 
대규모 데이터 세트와 여러 작업을 통해 훈련된 대규모 모델은 훈련되지 않은 특정 작업에도 잘 일반화됩니다. 이러한 모델을 파운데이션 모델(foundation model)이라고 하며, 스탠포드 HAI 연구소(Stanford Institute for Human-Centered Artificial Intelligence)에서 처음 대중화한 용어입니다. 
이러한 파운데이션 모델은 프롬프트 엔지니어링(prompt engineering) 기법의 도움으로 잘 활용할 수 있지만, 유스케이스가 도메인에 매우 특화되어 있거나 작업의 종류가 매우 다양하여 모델을 추가로 커스터마이징해야 하는 경우가 많습니다. 
특정 도메인이나 작업에 대한 대규모 모델의 성능을 개선하기 위한 한 가지 접근 방식은 더 작은 작업별 데이터 세트로 모델을 추가로 훈련하는 것입니다. 파인 튜닝(fine-tuning)으로 알려진 이 접근 방식은 LLM의 정확도를 성공적으로 개선하지만, 모든 모델 가중치를 수정해야 합니다. 
파인 튜닝 데이터 세트 크기가 훨씬 작기 때문에 사전 훈련(pre-training)하는 것 보다 훨씬 빠르지만 여전히 상당한 컴퓨팅 성능과 메모리가 필요합니다. 
파인 튜닝은 원본 모델의 모든 파라미터 가중치를 수정하므로 비용이 많이 들고 원본 모델과 동일한 크기의 모델을 생성하므로 스토리지 용량에도 부담이 됩니다.


### 응답:
LLM 분야의 발전과 함께 특정 작업이나 도메인에 대한 대규모 모델의 정확도를 개선하기 위해 파운데이션 모델과 파인튜닝 사이의 절충안이 제시

In [67]:
json_data

'{"generated_text": "\\uc544\\ub798\\ub294 \\uc791\\uc5c5\\uc744 \\uc124\\uba85\\ud558\\ub294 \\uba85\\ub839\\uc5b4\\uc640 \\ucd94\\uac00 \\ucee8\\ud14d\\uc2a4\\ud2b8\\ub97c \\uc81c\\uacf5\\ud558\\ub294 \\uc785\\ub825\\uc774 \\uc9dd\\uc744 \\uc774\\ub8e8\\ub294 \\uc608\\uc81c\\uc785\\ub2c8\\ub2e4. \\uc694\\uccad\\uc744 \\uc801\\uc808\\ud788 \\uc644\\ub8cc\\ud558\\ub294 \\uc751\\ub2f5\\uc744 \\uc791\\uc131\\ud558\\uc138\\uc694.\\n\\n### \\uba85\\ub839\\uc5b4:\\n\\ub2e4\\uc74c \\uae00\\uc744 \\uc54c\\uae30 \\uc27d\\uac8c \\uc694\\uc57d\\ud574 \\uc8fc\\uc138\\uc694.\\n\\n### \\uc785\\ub825:\\n\\n\\ub300\\uaddc\\ubaa8 \\uc5b8\\uc5b4 \\ubaa8\\ub378(LLM; Large Language Models) \\ubd84\\uc57c\\uc758 \\ubc1c\\uc804\\uacfc LLM\\uc774 \\uac00\\uce58 \\uc788\\ub294 \\uc778\\uc0ac\\uc774\\ud2b8\\ub97c \\uc81c\\uacf5\\ud558\\ub294 \\ubb38\\uc81c \\uc138\\ud2b8 \\uc218\\uac00 \\uacc4\\uc18d \\uc99d\\uac00\\ud558\\uace0 \\uc788\\ub2e4\\ub294 \\uc18c\\uc2dd\\uc744 \\ub4e4\\uc5b4\\ubcf4\\uc168\\uc744 \

In [71]:
generated_text = parse_response(json_data)

In [72]:
generated_text

'아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n다음 글을 알기 쉽게 요약해 주세요.\n\n### 입력:\n\n대규모 언어 모델(LLM; Large Language Models) 분야의 발전과 LLM이 가치 있는 인사이트를 제공하는 문제 세트 수가 계속 증가하고 있다는 소식을 들어보셨을 겁니다. \n대규모 데이터 세트와 여러 작업을 통해 훈련된 대규모 모델은 훈련되지 않은 특정 작업에도 잘 일반화됩니다. 이러한 모델을 파운데이션 모델(foundation model)이라고 하며, 스탠포드 HAI 연구소(Stanford Institute for Human-Centered Artificial Intelligence)에서 처음 대중화한 용어입니다. \n이러한 파운데이션 모델은 프롬프트 엔지니어링(prompt engineering) 기법의 도움으로 잘 활용할 수 있지만, 유스케이스가 도메인에 매우 특화되어 있거나 작업의 종류가 매우 다양하여 모델을 추가로 커스터마이징해야 하는 경우가 많습니다. \n특정 도메인이나 작업에 대한 대규모 모델의 성능을 개선하기 위한 한 가지 접근 방식은 더 작은 작업별 데이터 세트로 모델을 추가로 훈련하는 것입니다. 파인 튜닝(fine-tuning)으로 알려진 이 접근 방식은 LLM의 정확도를 성공적으로 개선하지만, 모든 모델 가중치를 수정해야 합니다. \n파인 튜닝 데이터 세트 크기가 훨씬 작기 때문에 사전 훈련(pre-training)하는 것 보다 훨씬 빠르지만 여전히 상당한 컴퓨팅 성능과 메모리가 필요합니다. \n파인 튜닝은 원본 모델의 모든 파라미터 가중치를 수정하므로 비용이 많이 들고 원본 모델과 동일한 크기의 모델을 생성하므로 스토리지 용량에도 부담이 됩니다.\n\n\n### 응답:\nLLM 분야의 발전과 함께 특정 작업이나 도메인에 대한 대규모 모델의 정확도를 개선하기 위해 파운데이션 모델과 파인튜닝 사이의 절충안이 제시

In [75]:
parts = generated_text.split('### 응답:')

# parts의 길이가 2 이상이면, '### 응답:' 이후의 부분을 추출합니다.
if len(parts) >= 2:
    response_text = parts[1]
else:
    response_text = ""  # '### 응답:'이 없는 경우 빈 문자열을 반환합니다.

print(response_text.strip())  # 앞뒤 공백을 제거하고 출력합니다.

LLM 분야의 발전과 함께 특정 작업이나 도메인에 대한 대규모 모델의 정확도를 개선하기 위해 파운데이션 모델과 파인튜닝 사이의 절충안이 제시되었습니다. 파인 튜닝 알고리즘을 이용하여 모델 학습 과정에서 일부 가중치와 특징을 추가로 수정할 수 있지만, 사전 훈련 과정보다 훨씬 빠르고 컴퓨팅 리소스와 메모리의 부담이 있습니다. 파운데이션 모델은 특정 작업에 대해 우수한 일반화를 제공하지만, 프롬프트 엔지니어링 기법이나 파인 튜닝 기법이 필요한 경우가 많습니다.<|endoftext|>
